# Convert Page 2030 Reporting Spreadsheet into DDX Bulk Upload File

### Import modules 

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)

# Set the display option to show all columns (None means no limit)
pd.set_option('display.max_columns', None)


In [ ]:
def column_check(x):
    if 'unnamed' in x.lower():
        return False
    if 'priority' in x.lower():
        return False
    if 'order' in x.lower():
        return True
    return True

In [ ]:
orderedColumns = ['Project name',
'Project number',
'Project status',
'Project type',
'Construction type',
'Office location',
'Country',
'State',
'City',
'Postal code',
'Climate zone',
'Use type 1',
'Use type area 1',
'Use type 2',
'Use type area 2',
'Unit',
'Estimated occupancy year',
'Energy code',
'Energy baseline',
'Energy baseline source',
'Energy target',
'Reporting phase',
'Reporting year',
'Net pEUI',
'Predicted LPD',
'Energy modeling tool',
'Other modeling tool name',
'Energy modeling party',
'Energy modeled date',
'Energy modeling cost',
'Predicted embodied carbon',
'Embodied carbon date',
'Carbon modeling tool',
'Other carbon modeling tool name',
'Carbon modeling tool version',
'Carbon modeling time',
'Carbon modeling scopes',
'Carbon modeling LCA stages',
'Has biogenic carbon',
'Biogenic carbon description',
'Onsite renewables',
'Window/Wall ratio',
'Grid electricity',
'Gas',
'District',
'Electricity from on-site renewables',
'Electricity from purchased off-site renewables',
'Electricity from dedicated off-site renewables']

### Import Data and Create dataframe

In [ ]:
data = pd.read_excel('ScriptingDataCopy.xlsx', sheet_name='Project Data', header=1, skiprows=[2,3,4,5], usecols=column_check, dtype=object)

#### Check Imported Data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# for col in data.columns:
#     print(col)

In [ ]:
data.shape

### Filter projects with `Reporting status` set to `Ready for DDX`

In [ ]:
ready_data = data[data['Reporting status']=='Ready for DDx']
ready_data.shape

In [ ]:
# ready_data[ready_data['Predicted LPD'] == 0]['Predicted LPD'] = None

### Correct Postal code

In [ ]:
# ready_data['Postal code'] = ready_data[(ready_data['Postal code']<5) & (ready_data['Country']=="United States")].astype(str).str.zfill(5)
ready_data.loc[ready_data['Country'] == 'United States', 'Postal code'] = ready_data.loc[ready_data['Country'] == 'United States', 'Postal code'].astype(str).str.zfill(5)

In [ ]:
# print(ready_data['Postal code'])

In [ ]:
def fixPostalCode(x):
    if len(x) < 5 and len(x) > 3:
        return '0'+x
    else:
        return x

In [ ]:
# ready_data['Postal code'] = ready_data['Postal code'].apply(str).map(fixPostalCode)
# ready_data['Energy modeled date'] = pd.to_datetime(ready_data['Energy modeled date'])

### Correct datetime

In [ ]:
ready_data['Energy modeled date'] = pd.to_datetime(ready_data['Energy modeled date'],errors='coerce')
ready_data['Energy modeled date'] = ready_data['Energy modeled date'].dt.date
# print(ready_data['Energy modeled date'])

In [ ]:
ready_data['Embodied carbon date'] = pd.to_datetime(ready_data['Embodied carbon date'], errors='coerce')
ready_data['Embodied carbon date'] = ready_data['Embodied carbon date'].dt.date
# print(ready_data['Embodied carbon date'])

### Remove Trailing Spaces

In [ ]:
ready_data['Country'] = ready_data['Country'].str.rstrip()
ready_data['State'] = ready_data['State'].str.rstrip()
ready_data['City'] = ready_data['City'].str.rstrip()

### Reorder dataframe to Match DDX Bulk Upload

In [ ]:
ready_data['Unit'] = 'IP'
# ready_data['Reporting year'] = 2022
# data = data.reindex(columns=orderedColumns)
ordered_data = ready_data[orderedColumns]

#### Check dataframe

In [ ]:
ordered_data.info()

In [ ]:
# ordered_data = ordered_data.drop(['Reporting status'], axis=1)

In [ ]:
ordered_data.head()

In [ ]:
ordered_data.shape

### Correct Gas Units to THERMS, rather than kBtu

In [ ]:
# ordered_data = ordered_data[ordered_data['Gas'] > 0]
ordered_data['Gas'] = pd.to_numeric(ordered_data['Gas'], errors='coerce')
# ordered_data.loc[ordered_data['Gas'] > 0] = ordered_data.loc[ordered_data['Gas'] > 0]/100
ordered_data['Gas'] = ordered_data['Gas']/100
ordered_data.shape

### Export dataframe as Excel File for Bulk Upload

In [ ]:
# ordered_data.to_excel('DDx Bulk Import 2023 Portfolio_20240415.xlsx', index=False)
ordered_data.to_excel('DDx Bulk Import 2023 Portfolio_20240415_Batch2.xlsx', index=False)